## LangChain OpeanAI Proxy

You may have access to a large langugage model through a HTTP Proxy like this one
https://github.com/derickson/ExpressOpenAIChatProxy

Assuming you know the URL for where it is hosted and the KEY for the proxy here is the code for using it.

your .env file will need this

```
PROXY_OPENAI_API_KEY="YOUR KEY"
PROXY_OPENAI_API_BASE="HTTPS://YOUR URL ENDING IN /v1"
```


In [1]:
! pip install -q openai langchain python_dotenv
import os
import openai
import langchain
from dotenv import load_dotenv

In [2]:
## OpenAI libraries like for either the environment variable or the openai.api_key to be set
load_dotenv(".env", override=True)
openai.api_key = os.environ["PROXY_OPENAI_API_KEY"]
openai.api_base = os.environ["PROXY_OPENAI_API_BASE"]

from langchain.chat_models import ChatOpenAI

default_model = "gpt-3.5-turbo"

llm = ChatOpenAI(
  openai_api_key=openai.api_key,
  openai_api_base=openai.api_base,
  temperature=0.3,
  model=default_model)

In [3]:
## Let's do a simple completion
llm.predict("The Prime Minister of the United Kingdom is ")

'Boris Johnson.'

In [5]:
retrieved_context = "The color of the sky is purple today"


system_prompt = f"""
You are a helpful AI assistant that answers questions using the provided context.
If you can't answer the question using the provided context, say "I don't know".
Don't apologize, and don't say you are an AI assistant.

Context: {retrieved_context}
"""

question = "What is the sky like today?"

messages = [
    {"role":"system", "content": system_prompt},
    {"role":"user", "content": question}
]

response = openai.ChatCompletion.create(
                        model=default_model,
                        max_tokens=100,
                        temperature=0,
                        messages=messages
                      )

response["choices"][0]["message"]["content"]

'The color of the sky is purple today.'